In [1]:
import pandas as pd
import mysql.connector

import datetime as dt

import sys
sys.path.append('../')
from src import soporte as sp

In [4]:
# cargamos el dataframe

df = pd.read_pickle('../datos/clima_2002-2022.pkl')

In [3]:
df.head()

,City,Latitude,Longitude,Date,Month,Year,Season,Temperature_(°C)_Max,Temperature_(°C)_Avg,Temperature_(°C)_Min,...,Dew_Point_(°C)_Min,Humidity_(%)_Max,Humidity_(%)_Avg,Humidity_(%)_Min,Wind_Speed_(kph)_Max,Wind_Speed_(kph)_Avg,Wind_Speed_(kph)_Min,Pressure_(hPa)_Max,Pressure_(hPa)_Avg,Pressure_(hPa)_Min
0,Elche,38.2851,-0.5625,2002-01-01,January,2002,winter,13.9,12.7,12.2,...,5.0,94.0,75.9,62.0,37.0,25.1,19.3,1023,1023,1019
1,Elche,38.2851,-0.5625,2002-01-02,January,2002,winter,13.9,12.7,10.0,...,7.2,82.0,76.7,67.0,40.2,23.0,4.8,1023,1019,1019
2,Elche,38.2851,-0.5625,2002-01-03,January,2002,winter,13.9,11.6,8.9,...,7.2,94.0,83.8,72.0,14.5,5.6,0.0,1019,1016,1013
3,Elche,38.2851,-0.5625,2002-01-04,January,2002,winter,15.0,12.5,8.9,...,6.1,94.0,77.6,63.0,29.0,12.9,3.2,1023,1016,1013
4,Elche,38.2851,-0.5625,2002-01-05,January,2002,winter,13.9,13.0,12.2,...,5.0,88.0,68.2,58.0,27.4,23.3,9.7,1029,1026,1023


Creación de la base de datos

In [5]:
# creamos la instancia del objeto de la clase Cargar

carga = sp.Cargar('climate', 'AlumnaAdalab')

In [8]:
# creamos la base de datos

carga.crear_bbdd()

La conexión a MySQL se ha realizado con exito.
MySQLCursor: CREATE DATABASE IF NOT EXISTS climate;


Creación de tablas

In [7]:
# creamos las queries para crear tablas

tabla_localidades = '''
                  CREATE TABLE IF NOT EXISTS `climate`.`locations` (
                  `id_location` INT NOT NULL AUTO_INCREMENT,
                  `latitude` DECIMAL(7,4) NULL,
                  `longitude` DECIMAL(7,4) NULL,
                  `city` VARCHAR(45) NOT NULL,
                  PRIMARY KEY (`id_location`))
                  ENGINE = InnoDB;
                    '''

tabla_estaciones = '''
              CREATE TABLE IF NOT EXISTS `climate`.`seasons` (
              `id_season` INT NOT NULL AUTO_INCREMENT,
              `season` VARCHAR(10) NOT NULL,
              PRIMARY KEY (`id_season`))
              ENGINE = InnoDB;
              '''

tabla_fechas = '''
              CREATE TABLE IF NOT EXISTS `climate`.`dates` (
              `id_date` INT NOT NULL AUTO_INCREMENT,
              `date` DATE NOT NULL,
              `seasons_id_season` INT NOT NULL,
              PRIMARY KEY (`id_date`),
              UNIQUE INDEX `date_UNIQUE` (`date` ASC) VISIBLE,
              INDEX `fk_date_seasons1_idx` (`seasons_id_season` ASC) VISIBLE,
              CONSTRAINT `fk_date_seasons1`
                FOREIGN KEY (`seasons_id_season`)
                REFERENCES `climate`.`seasons` (`id_season`)
                ON DELETE NO ACTION
                ON UPDATE NO ACTION)
              ENGINE = InnoDB;
              '''

tabla_mediciones = '''
              CREATE TABLE IF NOT EXISTS `climate`.`measurements` (
              `id_measurement` INT NOT NULL AUTO_INCREMENT,
              `temperature_max_C` FLOAT NULL,
              `temperature_avg_C` FLOAT NULL,
              `temperature_min_C` FLOAT NULL,
              `dew_point_min_C` FLOAT NULL,
              `humidity_max_%` FLOAT NULL,
              `humidity_avg_%` FLOAT NULL,
              `humidity_min_%` FLOAT NULL,
              `wind_speed_max_kph` FLOAT NULL,
              `wind_speed_avg_kph` FLOAT NULL,
              `wind_speed_min_kph` FLOAT NULL,
              `pressure_max_hpa` INT NULL,
              `pressure_avg_hpa` INT NULL,
              `pressure_min_hpa` INT NULL,
              `seasons_id_season` INT NOT NULL,
              `locations_id_location` INT NOT NULL,
              `date_id_date` INT NOT NULL,
              PRIMARY KEY (`id_measurement`),
              INDEX `fk_measurements_seasons_idx` (`seasons_id_season` ASC) VISIBLE,
              INDEX `fk_measurements_locations1_idx` (`locations_id_location` ASC) VISIBLE,
              INDEX `fk_measurements_date1_idx` (`date_id_date` ASC) VISIBLE,
              CONSTRAINT `fk_measurements_seasons`
                FOREIGN KEY (`seasons_id_season`)
                REFERENCES `climate`.`seasons` (`id_season`)
                ON DELETE NO ACTION
                ON UPDATE NO ACTION,
              CONSTRAINT `fk_measurements_locations1`
                FOREIGN KEY (`locations_id_location`)
                REFERENCES `climate`.`locations` (`id_location`)
                ON DELETE NO ACTION
                ON UPDATE NO ACTION,
              CONSTRAINT `fk_measurements_date1`
                FOREIGN KEY (`date_id_date`)
                REFERENCES `climate`.`dates` (`id_date`)
                ON DELETE NO ACTION
                ON UPDATE NO ACTION)
              ENGINE = InnoDB;
              '''

In [8]:
# creamos la tabla de estaciones

carga.crear_insertar_tabla(tabla_estaciones)


In [9]:
# creamos la tabla de fechas

carga.crear_insertar_tabla(tabla_fechas)

In [10]:
# creamos la tabla de localidades

carga.crear_insertar_tabla(tabla_localidades)

In [11]:
# creamos la tabla de mediciones

carga.crear_insertar_tabla(tabla_mediciones)

Insertamos los datos

In [12]:
query_localidades = f"""
        INSERT INTO locations (city, latitude, longitude) 
        VALUES ("Ottawa", 45.3202, -75.6682), ("Helsinki", 60.3179, 24.9496), ("Adelaide", -34.9462, 138.5312), ("Elche", 38.2851, -0.5625);
        """
carga.crear_insertar_tabla(query_localidades)

In [13]:
query_season = f"""
        INSERT INTO seasons (season) 
        VALUES ("summer"), ("winter"), ("spring"), ("autumn");
        """
carga.crear_insertar_tabla(query_season)

In [14]:
# creamos dataframes de las estaciones y localidades con ids

df['id_season'] = df['Season'].map({'summer' : 1, 'winter' : 2, 'spring' : 3, 'autumn' : 4})
df['id_location'] = df['City'].map({'Ottawa' : 1, 'Helsinki' : 2, 'Adelaide' : 3, 'Elche' : 4})

In [16]:
df.head()

,City,Latitude,Longitude,Date,Month,Year,Season,Temperature_(°C)_Max,Temperature_(°C)_Avg,Temperature_(°C)_Min,...,Humidity_(%)_Avg,Humidity_(%)_Min,Wind_Speed_(kph)_Max,Wind_Speed_(kph)_Avg,Wind_Speed_(kph)_Min,Pressure_(hPa)_Max,Pressure_(hPa)_Avg,Pressure_(hPa)_Min,id_season,id_location
0,Elche,38.2851,-0.5625,2002-01-01,January,2002,winter,13.9,12.7,12.2,...,75.9,62.0,37.0,25.1,19.3,1023,1023,1019,2,4
1,Elche,38.2851,-0.5625,2002-01-02,January,2002,winter,13.9,12.7,10.0,...,76.7,67.0,40.2,23.0,4.8,1023,1019,1019,2,4
2,Elche,38.2851,-0.5625,2002-01-03,January,2002,winter,13.9,11.6,8.9,...,83.8,72.0,14.5,5.6,0.0,1019,1016,1013,2,4
3,Elche,38.2851,-0.5625,2002-01-04,January,2002,winter,15.0,12.5,8.9,...,77.6,63.0,29.0,12.9,3.2,1023,1016,1013,2,4
4,Elche,38.2851,-0.5625,2002-01-05,January,2002,winter,13.9,13.0,12.2,...,68.2,58.0,27.4,23.3,9.7,1029,1026,1023,2,4


In [15]:
# creamos un dataframe de las fechas únicas

df_fechas = df.drop_duplicates(subset='Date')

In [16]:
# insertamos las fechas a la base de datos

for indice, fila in df_fechas.iterrows(): 
    
    query_date = f"""
            INSERT INTO dates (date, seasons_id_season) 
            VALUES ("{fila['Date']}", "{fila['id_season']}");
            """
    carga.crear_insertar_tabla(query_date)

In [18]:
# creamos la lista de columnas formateado para la query

columns = '''
`id_measurement` INT NOT NULL AUTO_INCREMENT,
`temperature_max_C` DECIMAL NULL,
`temperature_avg_C` DECIMAL NULL,
`temperature_min_C` DECIMAL NULL,
`dew_point_min_C` DECIMAL NULL,
`humidity_max_%` DECIMAL NULL,
`humidity_avg_%` DECIMAL NULL,
`humidity_min_%` DECIMAL NULL,
`wind_speed_max_kph` DECIMAL NULL,
`wind_speed_avg_kph` DECIMAL NULL,
`wind_speed_min_kph` DECIMAL NULL,
`pressure_max_hpa` INT NULL,
`pressure_avg_hpa` INT NULL,
`pressure_min_hpa` INT NULL,
`seasons_id_season` INT NOT NULL,
`locations_id_location` INT NOT NULL,
`date_id_date` INT NOT NULL,
'''

columns = columns.replace(' INT', '').replace(' NOT NULL', '').replace(' NULL', '').replace(' DECIMAL', '')

print(columns)


`id_measurement` AUTO_INCREMENT,
`temperature_max_C`,
`temperature_avg_C`,
`temperature_min_C`,
`dew_point_min_C`,
`humidity_max_%`,
`humidity_avg_%`,
`humidity_min_%`,
`wind_speed_max_kph`,
`wind_speed_avg_kph`,
`wind_speed_min_kph`,
`pressure_max_hpa`,
`pressure_avg_hpa`,
`pressure_min_hpa`,
`seasons_id_season`,
`locations_id_location`,
`date_id_date`,



In [19]:
# creamos la lista de columnas del dataframe formateado para la query

query_col = []

for col in df.columns:
    query_col.append(f"¿fila['{col}']?")
    
for i, e in enumerate(query_col):
    query_col[i] = e.replace('¿', '{').replace('?', '}')
    
query_col

["{fila['City']}",
 "{fila['Latitude']}",
 "{fila['Longitude']}",
 "{fila['Date']}",
 "{fila['Month']}",
 "{fila['Year']}",
 "{fila['Season']}",
 "{fila['Temperature_(°C)_Max']}",
 "{fila['Temperature_(°C)_Avg']}",
 "{fila['Temperature_(°C)_Min']}",
 "{fila['Dew_Point_(°C)_Max']}",
 "{fila['Dew_Point_(°C)_Avg']}",
 "{fila['Dew_Point_(°C)_Min']}",
 "{fila['Humidity_(%)_Max']}",
 "{fila['Humidity_(%)_Avg']}",
 "{fila['Humidity_(%)_Min']}",
 "{fila['Wind_Speed_(kph)_Max']}",
 "{fila['Wind_Speed_(kph)_Avg']}",
 "{fila['Wind_Speed_(kph)_Min']}",
 "{fila['Pressure_(hPa)_Max']}",
 "{fila['Pressure_(hPa)_Avg']}",
 "{fila['Pressure_(hPa)_Min']}",
 "{fila['id_season']}",
 "{fila['id_location']}"]

In [20]:
df.head()

,City,Latitude,Longitude,Date,Month,Year,Season,Temperature_(°C)_Max,Temperature_(°C)_Avg,Temperature_(°C)_Min,...,Humidity_(%)_Avg,Humidity_(%)_Min,Wind_Speed_(kph)_Max,Wind_Speed_(kph)_Avg,Wind_Speed_(kph)_Min,Pressure_(hPa)_Max,Pressure_(hPa)_Avg,Pressure_(hPa)_Min,id_season,id_location
0,Elche,38.2851,-0.5625,2002-01-01,January,2002,winter,13.9,12.7,12.2,...,75.9,62.0,37.0,25.1,19.3,1023,1023,1019,2,4
1,Elche,38.2851,-0.5625,2002-01-02,January,2002,winter,13.9,12.7,10.0,...,76.7,67.0,40.2,23.0,4.8,1023,1019,1019,2,4
2,Elche,38.2851,-0.5625,2002-01-03,January,2002,winter,13.9,11.6,8.9,...,83.8,72.0,14.5,5.6,0.0,1019,1016,1013,2,4
3,Elche,38.2851,-0.5625,2002-01-04,January,2002,winter,15.0,12.5,8.9,...,77.6,63.0,29.0,12.9,3.2,1023,1016,1013,2,4
4,Elche,38.2851,-0.5625,2002-01-05,January,2002,winter,13.9,13.0,12.2,...,68.2,58.0,27.4,23.3,9.7,1029,1026,1023,2,4


In [21]:
# creamos un query para extraer las fechas con las id

query_sacar_fechas = """
                    SELECT * FROM `dates`
                    """

In [22]:
# creamos un dataframe de las fechas con las id y convertimos las fechas a datetime

df_fechas_id = carga.devolver_datos('SELECT * FROM `dates`')
df_fechas_id['date'] = df_fechas_id['date'].apply(pd.to_datetime)

c:\Users\Master\Desktop\Adalab\climate-data-ETL\notebooks\..\src\soporte.py:123: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql_query(query, conexion)


In [23]:
# unimos esta dataframe de fechas con el dataframe de todo los datos para conseguir los id

df = df.merge(df_fechas_id, left_on='Date', right_on='date')

In [24]:
df

,City,Latitude,Longitude,Date,Month,Year,Season,Temperature_(°C)_Max,Temperature_(°C)_Avg,Temperature_(°C)_Min,...,Wind_Speed_(kph)_Avg,Wind_Speed_(kph)_Min,Pressure_(hPa)_Max,Pressure_(hPa)_Avg,Pressure_(hPa)_Min,id_season,id_location,id_date,date,seasons_id_season
0,Elche,38.2851,-0.5625,2002-01-01,January,2002,winter,13.9,12.7,12.2,...,25.1,19.3,1023,1023,1019,2,4,1,2002-01-01,2
1,Adelaide,-34.9462,138.5312,2002-01-01,January,2002,summer,18.9,16.2,13.9,...,33.8,25.7,1009,1006,1002,1,3,1,2002-01-01,2
2,Helsinki,60.3179,24.9496,2002-01-01,January,2002,winter,-8.9,-13.8,-20.0,...,10.3,0.0,1009,1002,999,2,2,1,2002-01-01,2
3,Ottawa,45.3202,-75.6682,2002-01-01,January,2002,winter,-7.8,-10.4,-13.9,...,17.7,8.0,1009,1006,999,2,1,1,2002-01-01,2
4,Elche,38.2851,-0.5625,2002-01-02,January,2002,winter,13.9,12.7,10.0,...,23.0,4.8,1023,1019,1019,2,4,2,2002-01-02,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30612,Ottawa,45.3202,-75.6682,2004-01-29,January,2004,winter,-10.0,-13.8,-16.1,...,23.5,11.3,996,992,992,2,1,7668,2004-01-29,1
30613,Adelaide,-34.9462,138.5312,2004-01-30,January,2004,summer,21.1,18.6,16.1,...,18.8,9.7,1013,1009,1009,1,3,7669,2004-01-30,1
30614,Ottawa,45.3202,-75.6682,2004-01-30,January,2004,winter,-13.9,-16.3,-17.8,...,19.8,12.9,992,765,0,2,1,7669,2004-01-30,1
30615,Adelaide,-34.9462,138.5312,2004-01-31,January,2004,summer,21.1,19.7,17.2,...,24.5,14.5,1009,1009,1009,1,3,7670,2004-01-31,1


In [25]:
# cargamos los datos de medidas a la base de datos 

for indice, fila in df.iterrows(): 
       
    query_measurement = f"""
            INSERT INTO measurements (`temperature_max_C`,
                                    `temperature_avg_C`,
                                    `temperature_min_C`,
                                    `dew_point_min_C`,
                                    `humidity_max_%`,
                                    `humidity_avg_%`,
                                    `humidity_min_%`,
                                    `wind_speed_max_kph`,
                                    `wind_speed_avg_kph`,
                                    `wind_speed_min_kph`,
                                    `pressure_max_hpa`,
                                    `pressure_avg_hpa`,
                                    `pressure_min_hpa`,
                                    `seasons_id_season`,
                                    `locations_id_location`,
                                    `date_id_date`) 
            VALUES (
                "{fila['Temperature_(°C)_Max']}",
                "{fila['Temperature_(°C)_Avg']}",
                "{fila['Temperature_(°C)_Min']}",
                "{fila['Dew_Point_(°C)_Min']}",
                "{fila['Humidity_(%)_Max']}",
                "{fila['Humidity_(%)_Avg']}",
                "{fila['Humidity_(%)_Min']}",
                "{fila['Wind_Speed_(kph)_Max']}",
                "{fila['Wind_Speed_(kph)_Avg']}",
                "{fila['Wind_Speed_(kph)_Min']}",
                "{fila['Pressure_(hPa)_Max']}",
                "{fila['Pressure_(hPa)_Avg']}",
                "{fila['Pressure_(hPa)_Min']}",
                "{fila['id_season']}",
                "{fila['id_location']}",
                "{fila['id_date']}");
            """
    carga.crear_insertar_tabla(query_measurement)